In [1]:
!pip install -U transformers datasets torch accelerate peft bitsandbytes sentencepiece
!pip install torch-pruning
!pip install git+https://github.com/huggingface/peft
!pip install einops

!apt-get update && apt-get install build-essential

files = ['shrek.txt' , 'shitposts.txt']

with open('merged.txt' , 'w') as outfile:
  for fname in files:
    try:
      with open(fname , 'r' , encoding = 'utf-8') as infile:
        outfile.write(infile.read() + "\n\n")
    except FileNotFoundError :
      print(f"[!] {fname} not found")
  print("text merged")

from transformers import GPT2Tokenizer , GPT2LMHeadModel , Trainer , TrainingArguments
from datasets import load_dataset

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token =tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2")

dataset = load_dataset('text' , data_files = 'merged.txt')
def tokenize_function(examples) :
  return tokenizer(examples["text"] , truncation = True , padding = True , max_length = 128)

tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=2)


!pip install -U bitsandbytes

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir="gpt2-shrek-chaos",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    save_steps=100,
    logging_steps=50,
    learning_rate=3e-4,
    weight_decay=0.01,
    fp16=True,
    logging_dir="logs",
    report_to="none",
    save_total_limit=1,

    disable_tqdm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
)

print("training..")
trainer.train()

model.save_pretrained("gpt2-shrek-chaos")
tokenizer.save_pretrained("gpt2-shrek-chaos")

print("saved to 'gpt2-shrek-chaos'")

import torch
from transformers import GPT2LMHeadModel, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = GPT2LMHeadModel.from_pretrained(
    "gpt2-shrek-chaos",
    device_map="auto",
    quantization_config=bnb_config,
    dtype=torch.float16,
)


import torch
import torch.nn as nn

model = GPT2LMHeadModel.from_pretrained("gpt2-shrek-chaos")
model.eval()
weighsToSparce = 0.5
for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        with torch.no_grad():
            w = module.weight.data
            mask = torch.rand(w.shape) > weighsToSparce
            w *= mask.to(w.device)

print("Model weights sparsified (" , weighsToSparce , ") zeroed out)")

model.save_pretrained("gpt2-shrek-chaos-sparse")

!pip install mistral-common


from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2-shrek-chaos")
tokenizer.save_pretrained("gpt2-shrek-chaos")

!rm -rf llama.cpp
!git clone https://github.com/ggerganov/llama.cpp
!pip install sentencepiece tokenizers gguf mistral-common

!cd llama.cpp && mkdir -p build && cd build && \
cmake .. \
-DCMAKE_BUILD_TYPE=Release \
&& make -j

!cp -r gpt2-shrek-chaos llama.cpp/models/gpt2-shrek-chaos

!cd llama.cpp && python3 convert_hf_to_gguf.py models/gpt2-shrek-chaos --outfile models/gpt2-shrek-chaos.gguf

!cd llama.cpp && ./build/bin/llama-quantize models/gpt2-shrek-chaos.gguf models/gpt2-shrek-chaos-Q2_K.gguf Q2_K

!ls -lh llama.cpp/models/*.gguf



!cd llama.cpp && ./build/bin/llama-cli -m models/gpt2-shrek-chaos-Q2_K.gguf -p "1+1 = " -n 100 --temp 1 --no-warmup --repeat_penalty 2.0

   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   --------------------------------- ------ 8.4/10.0 MB 52.1 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 44.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
   -- ------------------------------------- 12.6/199.4 MB 60.7 MB/s eta 0:00:04
   --- ------------------------------------ 15.7/199.4 MB 38.1 MB/s eta 0:00:05
   --- ------------------------------------ 18.4/199.4 MB 28.2 MB/s eta 0:00:07
   ---- ----------------------------------- 20.7/199.4 MB 24.7 MB/s eta 0:00:08
   ---- ----------------------------------- 23.3/199.4 MB 21.7 MB/s eta 0:00:09
   ----- ---------------------------------- 25.4/199.4 MB 20.1 MB/s eta 0:00:09
   ----- ---------------------------------- 27.5/199.4 MB 18.6 MB/s eta 0:00:10
   ----- ---------------------------------- 29.9/199.4 MB 17.7 MB/s eta 0:00:10
   ------ --------------------------------- 32.5/199.4 MB 17.1

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.1+cu118 requires torch==2.3.1+cu118, but you have torch 2.4.1 which is incompatible.
torchvision 0.18.1+cu118 requires torch==2.3.1+cu118, but you have torch 2.4.1 which is incompatible.


  Cloning https://github.com/huggingface/peft to c:\users\bscho\appdata\local\temp\pip-req-build-5lbiowmo


  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


[!] shrek.txt not found
[!] shitposts.txt not found
text merged


'apt-get' is not recognized as an internal or external command,
operable program or batch file.
C:\Users\bscho\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.models.gpt2.modeling_gpt2 because of the following error (look up to see its traceback):
operator torchvision::nms does not exist